---
  title: "RestFull 02: Jakarta RESTful Web Services"
  description: "Introduction to RestFull Web Services in Java with Jakarta RESTful Web Services."
  categories: 
    - Java
    - Lecture
    - RestFull
---

Java propose un standard appelé [Jakarta RESTful Web Services](https://jakarta.ee/specifications/restful-ws/3.1/jakarta-restful-ws-spec-3.1) pour construire efficacement des serveurs et des clients REST. 
[Jersey](https://eclipse-ee4j.github.io/jersey/download.html) est l'implantation de référence. 

## Une application REST minimale

Dans un premier temps nous allons étudier une application minimale qui s'appuie un serveur Web en Java [Grizzly](https://javaee.github.io/grizzly/).

L’archetype maven suivant permet de créer un projet de base dans le répertoire `/home/jovyan/work/src/samples/jaxrs/myresource`. 

In [1]:
%%shell
mkdir -p /home/jovyan/work/src/samples/jaxrs
cd /home/jovyan/work/src/samples/jaxrs
rm -rf /home/jovyan/work/src/samples/jaxrs/myresource

mvn archetype:generate --batch-mode --no-transfer-progress --quiet \
  -DarchetypeGroupId=org.glassfish.jersey.archetypes \
  -DarchetypeArtifactId=jersey-quickstart-grizzly2 \
  -DarchetypeVersion=3.1.5 \
  -DgroupId=fr.univtln.bruno.demos.jaxrs \
  -DartifactId=myresource

Le serveur peut être compilé puis exécuté 

```shell
cd /home/jovyan/work/src/samples/jaxrs/myresource
mvn package && mvn exec:java
```

Il est maintenant possible d’accéder à la ressource en ligne de commande à partir de l'adresse http://localhost:8080/myapp.


In [2]:
%%shell
cd /home/jovyan/work/src/samples/jaxrs/myresource
mvn --quiet --ntp package
nohup mvn --quiet --ntp exec:java &

Mar 05, 2024 1:09:49 PM org.glassfish.jersey.message.internal.MessagingBinders$EnabledProvidersBinder bindToBinder


Mar 05, 2024 1:09:49 PM org.glassfish.jersey.server.wadl.WadlFeature configure


Mar 05, 2024 1:09:49 PM org.glassfish.grizzly.http.server.NetworkListener start


INFO: Started listener bound to [localhost:8080]


Mar 05, 2024 1:09:49 PM org.glassfish.grizzly.http.server.HttpServer start


INFO: [HttpServer] Started.


Mar 05, 2024 1:09:49 PM org.glassfish.grizzly.http.server.NetworkListener shutdownNow


INFO: Stopped listener bound to [localhost:8080]


Une application REST JAX-RS est construite autour de deux notions principales l'Application (le serveur) et les Ressources.
Une instance d'une ressource est créée pour répondre à chaque requête et détruite ensuite. Elle peut donc être utilisée comme une sorte de singleton. Une ressource est définie annotant  une classe, un ou plusieurs de ses superclasses (y compris abstraites) ou l'une de ses interfaces. 

Dans l'exemple, la classe `fr.univtln.bruno.demos.jaxrs.Main` démarre le serveur et paramètre les packages où le framework va chercher des ressources comme le montre la méthode ci-dessous.

In [3]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/samples/jaxrs/myresource/src/main/java/fr/univtln/bruno/demos/jaxrs/Main.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("Main","startServer"),script);
return null;

```Java
/**
 * Starts Grizzly HTTP server exposing JAX-RS resources defined in this application.
 * @return Grizzly HTTP server.
 */
public static HttpServer startServer() {
    // create a resource config that scans for JAX-RS resources and providers
    // in fr.univtln.bruno.demos.jaxrs package
    final ResourceConfig rc = new ResourceConfig().packages("fr.univtln.bruno.demos.jaxrs");
    // create and start a new instance of grizzly http server
    // exposing the Jersey application at BASE_URI
    return GrizzlyHttpServerFactory.createHttpServer(URI.create(BASE_URI), rc);
}
```

La classe `fr.univtln.bruno.demos.jaxrs.MyResource` présente le fonctionnement minimal d'une ressource. La classe est annotée avec `@Path(...)` pour indiquer le chemin à ajouter à l’URL correspondant à cette ressource. D'une façon générale, les méthodes sont annotées avec `@POST`, `@GET`, `@PUT`,`@DELETE`, ... pour indiquer le type de verbe HTTP associé. 

Les méthodes peuvent être annotées avec `@Produces` qui indique le ou les types MIME dans lequel le résultat peut être fourni : `@Produces("text/plain")`, `@Produces("application/json")`, … Il est possible d’indiquer plusieurs types avec `@Produces({"application/json", "application/xml"})`. Il existe aussi des constantes équivalentes `MediaType.TEXT_PLAIN`. Une valeur par défaut de `@Produces` peut être indiquée en annotant la classe.

In [4]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/samples/jaxrs/myresource/src/main/java/fr/univtln/bruno/demos/jaxrs/MyResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByAnnotationName",List.of("MyResource","GET"),script);
return null;

```Java
/**
 * Method handling HTTP GET requests. The returned object will be sent
 * to the client as "text/plain" media type.
 *
 * @return String that will be returned as a text/plain response.
 */
@GET
@Produces(MediaType.TEXT_PLAIN)
public String getIt() {
    return "Got it!";
}
```

Le client peut indiquer le type demandé parmi l'un des "produces" avec la valeur `Content-Type: ` de l'entête de la requête.


La commande suivante exécute une requête GET sur l'URL d'une ressource et affiche le résultat en-tête compris. (Le serveur doit être lancé).

In [5]:
%%shell
curl -s -D - --get http://localhost:8080/myresource

## Une application REST plus complète

Pour la suite nous allons étudier en détail l'application https://github.com/ebpro/sample-jaxrs.

In [6]:
//| output: false
//| echo: false

%printWithName false
    
// UPDATE SAMPLE SOURCE CODE

String script="""
GITHUB_REPO=ebpro/sample-jaxrs
GITHUB_URL=https://github.com/${GITHUB_REPO}
BRANCH=develop
SRC_DIR=/home/jovyan/work/src/github/${GITHUB_REPO}
gitpuller ${GITHUB_URL} ${BRANCH} ${SRC_DIR}
cd ${SRC_DIR}
mvn --quiet clean package
""";
IJava.getKernelInstance().getMagics().applyCellMagic("shell",List.of(""),script);  

Mar 05, 2024 1:09:51 PM org.glassfish.jersey.message.internal.MessagingBinders$EnabledProvidersBinder bindToBinder


Mar 05, 2024 1:09:51 PM org.glassfish.jersey.server.wadl.WadlFeature configure


Mar 05, 2024 1:09:51 PM org.glassfish.grizzly.http.server.NetworkListener start


INFO: Started listener bound to [localhost:8080]


Mar 05, 2024 1:09:51 PM org.glassfish.grizzly.http.server.HttpServer start


...

[ERROR] Re-run Maven using the -X switch to enable full debug logging.


[ERROR] 


[ERROR] For more information about the errors and possible solutions, please read the following articles:


[ERROR] [Help 1] http://cwiki.apache.org/confluence/display/MAVEN/MojoExecutionException


IJava.getKernelInstance().getMag...: 1

L'application peut être compilée et exécutée avec maven.
Cela lance le serveur (vous pourrez l'arrêter avec ctrl-c dans le terminal).
Cela compile, exécute les tests unitaires, package et exécute les tests d'intégration (en lançant le serveur REST et en exécutant de vraies requêtes).

```shell 
cd /home/jovyan/work/src/github/ebpro/sample-jaxrs && \
    mvn clean verify &&
    mvn exec:java
```

La classe `fr.univtln.bruno.samples.jaxrs.server.BiblioServer` paramètre Jersey, démarre Grizzly et attend un CTRL-C pour arrêter le serveur. 

La classe `fr.univtln.bruno.samples.jaxrs.model.LibraryModel` définit le modèle de donnée (Une bibliothèque qui est une facade pour gérer des Auteurs et des Livres.)

Les classes `fr.univtln.bruno.samples.jaxrs.resources.LibraryResource` et `fr.univtln.bruno.samples.jaxrs.resources.AuthorResource` définissent des ressources REST.

In [7]:
//| output: false
//| echo: false
%jars "/home/jovyan/work/src/github/ebpro/sample-jaxrs/target/sample-jaxrs-*-withdependencies.jar"; 
import org.glassfish.grizzly.http.server.HttpServer;
import fr.univtln.bruno.samples.jaxrs.server.BiblioServer;
HttpServer httpServer = BiblioServer.startServer();
httpServer.toString();

CompilationException: 

### Chemins et Verbes
La méthode `sayHello()` reprend l'exemple précédent.

In [8]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/LibraryResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("LibraryResource","sayHello"),script);
return null;

```Java
/**
 * The simpliest method that just return "hello" in plain text with GET on the default path "biblio".
 *
 * @return the string
 */
@SuppressWarnings("SameReturnValue")
@GET
@Path("hello")
@Produces(MediaType.TEXT_PLAIN)
public String sayHello() {
    return "hello";
}
```

In [9]:
%%shell
curl -s -v http://localhost:9998/mylibrary/library|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


D'autres verbe peuvent être utilisé sur le même chemin. Une méthode peut aussi être annotées avec `@Path` pour définir le chemin associé à cette méthode.

La méthode `init()` est un simple `PUT` sans paramètre qui initialise la bibliothèque avec deux auteurs.

In [10]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/LibraryResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("LibraryResource","init"),script);
return null;

```Java
/**
 * An init method that add two authors with a PUT on the default path.
 *
 * @return the number of generated authors.
 * @throws IllegalArgumentException the illegal argument exception
 */
@PUT
@Path("init")
public int init() throws BusinessException {
    Library.demoLibrary.removesAuthors();
    Library.Author author1 = Library.demoLibrary.addAuthor(Library.Author.builder().firstname("Alfred").name("Martin").build());
    Library.Author author2 = Library.demoLibrary.addAuthor(Library.Author.builder().firstname("Marie").name("Durand").build());
    Library.demoLibrary.addBook(Library.Book.builder().title("title1").authors(Set.of(author1)).build());
    Library.demoLibrary.addBook(Library.Book.builder().title("title2").authors(Set.of(author1, author2)).build());
    Library.demoLibrary.addBook(Library.Book.builder().title("title3").authors(Set.of(author2)).build());
    Library.demoLibrary.addBook(Library.Book.builder().title("title4").authors(Set.of(author2)).build());
    return Library.demoLibrary.getAuthorsNumber();
}
```

In [11]:
%%shell
curl -s -i -X PUT "http://localhost:9998/mylibrary/library/init"

### Les paramètres simples
JAX-RS permet d'extraire automatiquement des valeurs de paramètres depuis le chemin de la ressources, les paramêtres de la requête ou l'entête http. Ces valeurs peuvent alors être "injectées" (affectée par annotation aux paramètres des méthodes REST).

L’annotation `@PathParam` permet d’injecter les valeurs provenant des URL comme des paramètres.

La méthode `getAuthor(@PathParam("id") final long id)` ci dessous-s'exécute lors d'un `GET` sur un chemin de forme `@Path("author/{id}")`. `id` est est un pas de chemin quelconque qui sera extrait, converti en long et injecté grâce à `@PathParam` dans le paramètre `id` de la fonction. Il est possible d'indiquer une expression régulière pour contraindre la forme du pas par exemple `@Path("authors/{id: [0-9]+}")`. 

Le `@Produces` sur la classe indique que du XML ou du JSON peuvent être produits.
Les méthodes REST retournent instance de la classe Response qui représente une réponse HTTP. Cette classe propose un builder pour construire manuellement. 
Cependant, JAX-RS permet de construite automatiquement ces réponses si le type de retour peut être transformé en un contenu de réponse (une entité http) par une implantation de l'interface `MessageBodyWriter`. Les implantations de JAX-RS en fournissent généralement par défaut par exemple pour String voire pour XML ou JSON (via les mécanismes de marshalling qui seront étudiés en détail plus tard).

In [12]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","getAuthor"),script);
return null;

```Java
/**
 * Find and return an author by id with a GET on the path "biblio/auteurs/{id}" where  {id} is the needed id.
 * The path parameter "id" is injected with @PathParam.
 *
 * @param id the needed author id.
 * @return the auteur with id.
 * @throws NotFoundException is returned if no author has the "id".
 */
@GET
@Path("{id}")
public Library.Author getAuthor(@PathParam("id") final long id) throws BusinessException {
    return Library.demoLibrary.getAuthor(id);
}
```

Get author 1 in JSON :

In [13]:
%%shell
curl -s -v -H "Accept: application/json"  \
  http://localhost:9998/mylibrary/authors/1|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


La requête suivante reprend la précédente et demande du XML. JAX-RS va chercher automatiquement des classes (MessageBodyWriter et Reader) pour créer le bon format. Ces classes peuvent construites explicitement mais des extensions peuvent être ajoutées pour produire les types classiques par annotations des entités (cf. le pom.xml et  les annotations de la classe `BiblioModel.Auteur`) : jersey-media-jaxb pour XML et jersey-media-json-jackson pour JSON. Jackson n'est pas l'implantatation pas défaut mais elle est plus efficace et plus configurable. 

Get author 2 in XML :

In [14]:
%%shell
curl -s -i -H "Accept: text/xml"  \
  http://localhost:9998/mylibrary/authors/2

Les collections classiques sont supportés. Notez qu'ici les [collections eclipse](https://www.eclipse.org/collections/) sont utilisées en particulier celles pour les primitifs et qu'elles sont supportées par Jackson.

Get authors in XML :

In [15]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","getAuthors"),script);
return null;

```Java
/**
 * Gets auteurs.
 *
 * @return the auteurs
 */
@GET
public Collection<Library.Author> getAuthors() {
    return Library.demoLibrary.getAuthors().values();
}
```

Get authors in JSON

In [16]:
%%shell
curl -s -v -H "Accept: application/json"  \
  http://localhost:9998/mylibrary/authors|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


D'une façon similaire les annotations `@HeaderParam` et `@QueryParam` permettent d'extraire des valeurs de l'entête ou des paramètres de la requête http.
La méthode suivante permet de construire un filtre pour des requêtes complexe. L'utilisation d'un chemin différent ("filter") n'est utile que pour l'exemple dans une application réelle il n'y aura qu'un seul GET. 

In [17]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","getFilteredAuthors"),script);
return null;

```Java
/**
 * Gets a list of "filtered" authors.
 *
 * @param name        an optional exact filter on the name.
 * @param firstname     an optional exact filter on the firstname.
 * @param biography an optional contains filter on the biography.
 * @param sortKey    the sort key (prenom or nom).
 * @return the filtered auteurs
 */
@GET
@Path("filter")
public Page<Library.Author> getFilteredAuthors(@QueryParam("name") String name, @QueryParam("firstname") String firstname, @QueryParam("biography") String biography, @HeaderParam("sortKey") @DefaultValue("name") String sortKey) {
    PaginationInfo paginationInfo = PaginationInfo.builder().name(name).firstname(firstname).biography(biography).sortKey(sortKey).build();
    return Library.demoLibrary.getAuthorsWithFilter(paginationInfo);
}
```

In [18]:
%%shell
curl -s -v -H "Accept: application/json"  \
 "http://localhost:9998/mylibrary/authors/filter?name=Durand&firstname=Marie"|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


In [19]:
%%shell
curl -s -v -H "Accept: application/json"  \
-H "sortKey: firstname" \
  "http://localhost:9998/mylibrary/authors/filter"|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


Pour simplifier le traitement des paramètres JAX-RX propose l'annotation `@BeanParam` qui permet de créer un instance d'une classe à partir des paramètres extraits. Pour cela, les propriétés de la classe peuvent être annotées pour indiquer les paramêtres correspondants.

L'exemple suivant montre comment l'utiliser pour mettre en place la pagination qui est essentielle quand le volume des données peut être important. Là aussi le chemin spécifique ("page") n'est là que pour l'exemple. 

In [20]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/PaginationInfo.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcClassByName",List.of("PaginationInfo"),script);
return null;

```Java
@FieldDefaults(level = AccessLevel.PRIVATE)
@Getter
@ToString
@Builder
@NoArgsConstructor
@AllArgsConstructor
public class PaginationInfo {
    @SuppressWarnings("FieldMayBeFinal")
    @QueryParam("page")
    @Builder.Default
    long page = 1;

    @SuppressWarnings("FieldMayBeFinal")
    @QueryParam("pageSize")
    @Builder.Default
    long pageSize = 10;

    @HeaderParam("sortKey")
    @DefaultValue("name")
    String sortKey;

    @QueryParam("name")
    String name;

    @QueryParam("firstname")
    String firstname;

    @QueryParam("biography")
    String biography;
}
```

In [21]:
//| output: true
//| echo: false
// PRINT CLASS
String script="/home/jovyan/work/src/github/ebpro/sample-jaxrs/src/main/java/fr/univtln/bruno/samples/jaxrs/resources/AuthorResource.java";
IJava.getKernelInstance().getMagics().applyCellMagic("javasrcMethodByName",List.of("AuthorResource","getAuthorsPage"),script);
return null;

```Java
/**
 * Gets a page of authors after applying a sort.
 *
 * @param paginationInfo the pagination info represented as a class injected with @BeanParam.
 * @return the page of authors.
 */
@GET
@Path("page")
public Page<Library.Author> getAuthorsPage(@BeanParam PaginationInfo paginationInfo) {
    return Library.demoLibrary.getAuthorsWithFilter(paginationInfo);
}
```

L'appel suivant de l'API génère alétoirement 100 auteurs.

In [22]:
%%shell
curl -s -i -X PUT "http://localhost:9998/mylibrary/library/init/100"

On peut alors demander la page 3 (de taille 4).

In [23]:
%%shell
curl -s -v -H "Accept: application/json"  \
-H "sortKey: firstname" \
  "http://localhost:9998/mylibrary/authors/page?pageSize=4&page=3"|jq

*   Trying 127.0.0.1:9998...


* connect to 127.0.0.1 port 9998 failed: Connection refused


*   Trying ::1:9998...


* Immediate connect fail for ::1: Cannot assign requested address


* Failed to connect to localhost port 9998 after 0 ms: Connection refused


* Closing connection 0


L'appel suivant de l'API remet uniquement deux auteurs dans la base de données.

In [24]:
%%shell
curl -s -i -X PUT "http://localhost:9998/mylibrary/library/init"

In [25]:
httpServer.stop();

CompilationException: 